In [10]:
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 21 22:41:12 2020

@author: GUR45397
"""

import tensorflow as tf
from skimage import io
import numpy as np
import time
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

import os

trainingDataPath = "datasets/mnist/trainingSample/"
testingDataPath = "datasets/mnist/testset/" 
height = 28
width = 28
channels = 1



In [11]:
mean = 0
var = 0.1
sigma = var**0.5
gaussisanNoise = np.random.normal(mean,sigma,(height,width,channels))
gaussisanNoise = gaussisanNoise.reshape(height,width,channels)



In [12]:
def get_batch(batch_size, forTraining):
    if(True == forTraining):
        path = trainingDataPath
    else:
        path = testingDataPath
    imagesName = os.listdir(path)
    for batch_i in range(0, len(imagesName)//batch_size):
        start_i = batch_i * batch_size
        try:
            batch = imagesName[start_i:start_i + batch_size]           
        except IndexError:
            batch = imagesName[start_i:]
        noisedImageList = []
        actualImageList = []
        for image in batch:
            imageData = io.imread(path+'/'+image)
            imageData = imageData/255
            imageData = imageData.reshape(width,height,channels)
            
            actualImageList.append(imageData)
            noisedImageList.append(imageData+gaussisanNoise)
        yield(np.array(noisedImageList), np.array(actualImageList))
        
#for i, (X_image,angle) in enumerate(get_batch(100)):
#    print(i)
    


In [13]:

        



tf.compat.v1.reset_default_graph()


inputVal = tf.compat.v1.placeholder(tf.float32, shape = [None ,height,width,channels],name = 'features')
output = tf.compat.v1.placeholder(tf.float32, shape = [None,height,width,channels],name = 'output')
lr = tf.compat.v1.placeholder(tf.float32, name = 'lr')



### Encoder
conv1 = tf.layers.conv2d(inputs=inputVal, filters=32, kernel_size=(3,3), padding='same', activation=tf.nn.relu, name="convLayer1")

print(conv1)
maxpool1 = tf.layers.max_pooling2d(conv1, pool_size=(2,2), strides=(2,2), padding='same')
conv2 = tf.layers.conv2d(inputs=maxpool1, filters=32, kernel_size=(3,3), padding='same', activation=tf.nn.relu, name="convLayer2")
maxpool2 = tf.layers.max_pooling2d(conv2, pool_size=(2,2), strides=(2,2), padding='same')
print(conv2, maxpool2)

conv3 = tf.layers.conv2d(inputs=maxpool2, filters=16, kernel_size=(3,3), padding='same', activation=None, name="convLayer3")
encoded = tf.layers.max_pooling2d(tf.nn.relu(conv3), pool_size=(2,2), strides=(2,2), padding='same')
midlayer = tf.identity(conv3,"bottle_neck_layer")
### Decoder
upsample1 = tf.image.resize_images(encoded, size=(7,7), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
conv4 = tf.layers.conv2d(inputs=upsample1, filters=16, kernel_size=(3,3), padding='same', activation=tf.nn.relu)
upsample2 = tf.image.resize_images(conv4, size=(14,14), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
conv5 = tf.layers.conv2d(inputs=upsample2, filters=32, kernel_size=(3,3), padding='same', activation=tf.nn.relu)
upsample3 = tf.image.resize_images(conv5, size=(28,28), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
conv6 = tf.layers.conv2d(inputs=upsample3, filters=32, kernel_size=(3,3), padding='same', activation=tf.nn.relu)
decoded = tf.layers.conv2d(inputs=conv6, filters=1, kernel_size=(3,3), padding='same', activation=None)

residual_error = tf.pow(decoded-inputVal,2)
cost = tf.reduce_mean(residual_error) 


optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(cost, name="optimizer")




Tensor("convLayer1/Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("convLayer2/Relu:0", shape=(?, 14, 14, 32), dtype=float32) Tensor("max_pooling2d_1/MaxPool:0", shape=(?, 7, 7, 32), dtype=float32)


In [14]:
epoch=10

init = tf.compat.v1.global_variables_initializer()
with tf.compat.v1.Session() as sess:
    summ_writer = tf.compat.v1.summary.FileWriter("out",sess.graph)
    init.run()
    stime = time.time()
    saver = tf.train.Saver()
    
    for i in range(epoch):
        average_loss = []
        for batch_index, (noisedImage, actualImage) in enumerate(get_batch(
                                            batch_size=100, forTraining=True)):
            _,loss = sess.run((train_op, cost), 
                                 feed_dict={inputVal:noisedImage, 
                                                output:actualImage,
                                                lr:0.001})
            average_loss.append(loss)
            
        summary = tf.Summary()
        summary.value.add(tag="Trainloss", simple_value=np.mean(average_loss))

        summ_writer.add_summary(summary, i)
        summ_writer.flush()

        for batch_index, (noisedImage, actualImage) in enumerate(get_batch(
                                        batch_size=100, forTraining=False)):
            m, loss = sess.run((midlayer, cost), 
                                 feed_dict={inputVal:noisedImage})
               


            summary = tf.Summary()
            summary.value.add(tag="Validloss", simple_value=loss)

            summ_writer.add_summary(summary, i)
            summ_writer.flush()
            print("******************Train:",np.mean(average_loss), "test:",loss,"i:", i) 
            break
    #print(tf.trainable_variables())        
    c1_w = tf.trainable_variables()[0].eval()
    c2_w = tf.trainable_variables()[2].eval()
    c3_w = tf.trainable_variables()[4].eval()
    
    print(tf.trainable_variables()[1].eval())
    saver.save(sess,"out/graph/tensorflowModel.ckpt")
    tf.train.write_graph(sess.graph.as_graph_def(), "out/graph", 'tensorflowModel.pbtxt', as_text=True)
    

******************Train: 0.18752186 test: 0.17590873 i: 0
******************Train: 0.1711989 test: 0.16605169 i: 1
******************Train: 0.16368745 test: 0.1606621 i: 2
******************Train: 0.15900376 test: 0.15598531 i: 3
******************Train: 0.15495028 test: 0.1519406 i: 4
******************Train: 0.15093909 test: 0.14795493 i: 5
******************Train: 0.14687854 test: 0.14372356 i: 6
******************Train: 0.1428446 test: 0.14040484 i: 7
******************Train: 0.13938662 test: 0.13657524 i: 8
******************Train: 0.13547833 test: 0.1321939 i: 9
[ 3.6320342e-03 -4.4149416e-03 -1.1061602e-05  5.5956347e-03
  6.1686570e-03 -2.6084653e-03  3.1318343e-03  6.2185284e-03
 -8.3545683e-04  3.7256642e-03 -2.7629605e-04 -5.5981535e-03
  1.4219645e-03  7.4025029e-03  4.3555531e-03 -1.8524997e-03
 -4.9785725e-03 -5.5164807e-03  1.9329223e-03  1.1189973e-02
  1.4510464e-02  1.0013958e-02 -4.2620869e-03  3.6067588e-03
 -4.4161984e-04 -8.5923700e-03  2.1126161e-03  5.8145835e-0

In [15]:
freeze_graph.freeze_graph('out/graph/tensorflowModel.pbtxt', "", 
                              False, 
                              'out/graph/tensorflowModel.ckpt', 
                              "features, bottle_neck_layer, optimizer", #Since using optimizer, as well it will have
                              #the full graph details thats why size would be as same as "out/graph/tensorflowModel.ckpt'"
                              "save/restore_all", "save/Const:0", #not relevant
                              'out/graph/frozentensorflowModel.pb', 
                              True, ""  
                         )
    
inputGraph = tf.GraphDef()
with tf.gfile.Open("out/graph/frozentensorflowModel.pb","rb") as f:
    data2read = f.read()
    inputGraph.ParseFromString(data2read)
outputGraph = optimize_for_inference_lib.optimize_for_inference(
                inputGraph,
                ["features"],
                ["bottle_neck_layer"],#optimizer not considered, size is small.
                tf.float32.as_datatype_enum)
f = tf.gfile.FastGFile('out/graph/OptimizedGraph.pb', "w")
f.write(outputGraph.SerializeToString()) 
    
        
        
        
            

INFO:tensorflow:Restoring parameters from out/graph/tensorflowModel.ckpt
INFO:tensorflow:Froze 44 variables.
INFO:tensorflow:Converted 44 variables to const ops.


In [26]:

 %reload_ext tensorboard

In [27]:
%tensorboard --logdir "out" --port 1231

Reusing TensorBoard on port 1231 (pid 1124), started 0:00:27 ago. (Use '!kill 1124' to kill it.)

In [28]:
tf.reset_default_graph()
def load_graph():
    graph = tf.Graph()
    graph_def = tf.compat.v1.GraphDef()
    with open("out/graph/OptimizedGraph.pb","rb") as f:
        graph_def.ParseFromString(f.read())
    
    with graph.as_default():
        tf.graph_util.import_graph_def(graph_def)
        
        #tf.import_graph_def(graph_def)
    
    return graph

graph = load_graph()



In [30]:
for i, (X_image,angle) in enumerate(get_batch(1, False)):
    print()
    break
input_name = "import/features"


output_Encoder_features_op = "import/bottle_neck_layer"

input_operation = graph.get_operation_by_name(input_name);

output_Encoder_features = graph.get_operation_by_name(output_Encoder_features_op);

with tf.compat.v1.Session(graph=graph) as sess:
    results = sess.run(output_Encoder_features.outputs[0],
                      {input_operation.outputs[0]: X_image})
    print(results.shape)


(1, 7, 7, 16)


In [31]:
import pandas as pd

trainingDataPathforclass = "datasets/mnist/mnist_train.csv"
testingDataPathforclass = "datasets/mnist/mnist_test.csv" 

def get_batch_for_classification(batch_size, forTraining):
    if(True == forTraining):
        path = trainingDataPathforclass
    else:
        path = testingDataPathforclass
    
    df = pd.read_csv(path)
    df = df[0:1000]
    labels = df['label']
    imageData = df.iloc[:,1:]
    imageBatch = []
    for batch_i in range(0, len(labels)//batch_size):
        start_i = batch_i * batch_size
        try:
            batch = imageData[start_i:start_i + batch_size]  
            batch_l = labels[start_i:start_i + batch_size]
        except IndexError:
            batch = imageData[start_i:]
            batch_l = labels[start_i:]
         
        batch = batch/255
        batch = np.array(batch)
        batch =  batch.reshape(batch_size,28,28,1)+gaussisanNoise
        #batch = batch.reshape(batch_size,28,28,1)
        yield(batch, np.array(batch_l))


In [32]:

output = 10
h = 28
w = 28
c = 1

tf.reset_default_graph()
encoded_input =  tf.compat.v1.placeholder(tf.float32, shape = [None, h, w, c],name = 'encoded_input')
label = tf.compat.v1.placeholder(tf.int32, shape = [None],name = 'label')
rate = tf.compat.v1.placeholder(tf.float32, name = 'rate')

In [33]:

filter1 = tf.compat.v1.get_variable('weights11', initializer= c1_w, dtype=tf.float32, trainable=False)    
conv1 = tf.nn.conv2d(encoded_input, filter1, strides = 1, padding='SAME', name = 'conv1')
conv1 = tf.nn.relu(conv1)
maxpool1 = tf.layers.max_pooling2d(conv1, pool_size=(2,2), strides=(2,2), padding='same')

filter2 = tf.compat.v1.get_variable('weights12',initializer= c2_w,dtype=tf.float32,trainable=False)    
conv2 = tf.nn.conv2d(maxpool1, filter2, strides = 1, padding='SAME', name = 'conv2')
conv2 = tf.nn.relu(conv2)
maxpool2 = tf.layers.max_pooling2d(conv2, pool_size=(2,2), strides=(2,2), padding='same')

filter3 = tf.compat.v1.get_variable('weights13',initializer= c3_w,dtype=tf.float32, trainable=False)    
conv3 = tf.nn.conv2d(maxpool2, filter3, strides = 1, padding='SAME', name = 'conv3')
conv3 = tf.nn.relu(conv3)

conv3_reshaped = tf.reshape(conv3,shape = (-1,conv3.shape[1]*conv3.shape[2]*conv3.shape[3]))


dense1  = tf.compat.v1.layers.dense(conv3_reshaped,512,activation=tf.nn.relu)
dense2  = tf.compat.v1.layers.dense(dense1,128,activation=tf.nn.relu)
logits  = tf.compat.v1.layers.dense(dense2,output,name="logits")

In [34]:
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label, logits=logits)
loss = tf.reduce_mean(xentropy)

In [35]:
gd = tf.compat.v1.train.AdamOptimizer(learning_rate=rate)
op = gd.minimize(loss, name="GD")

In [44]:
epoch=10

init = tf.compat.v1.global_variables_initializer()
with tf.compat.v1.Session() as sess:
    summ_writer = tf.compat.v1.summary.FileWriter("out2",sess.graph)
    init.run()
    stime = time.time()
    saver = tf.train.Saver()
    
    for i in range(epoch):
        average_loss = []
        for batch_index, (noisedImage, labels) in enumerate(get_batch_for_classification(
                                            batch_size=100, forTraining=True)):
            _,l = sess.run((op, loss), 
                                 feed_dict={encoded_input:noisedImage, 
                                                label:labels,
                                                rate:0.001})
            #print(l)
            average_loss.append(l)
            
        summary = tf.Summary()
        summary.value.add(tag="CTrainloss", simple_value=np.mean(average_loss))

        summ_writer.add_summary(summary, i)
        summ_writer.flush()

        for batch_index, (noisedImage, actualImage) in enumerate(get_batch_for_classification(
                                            batch_size=100, forTraining=False)):
            l = sess.run(loss, feed_dict={encoded_input:noisedImage,label:labels})
               


            summary = tf.Summary()
            summary.value.add(tag="CValidloss", simple_value=l)

            summ_writer.add_summary(summary, i)
            summ_writer.flush()
            print("******************Train:",np.mean(average_loss), "test:",l,"i:", i) 
            break
    saver.save(sess,"out2/graph/tensorflowModel2.ckpt")
    tf.train.write_graph(sess.graph.as_graph_def(), "out2/graph", 'tensorflowModel2.pbtxt', as_text=True)
    

******************Train: 2.0400944 test: 2.4007676 i: 0
******************Train: 1.4604642 test: 2.7808564 i: 1
******************Train: 1.0766575 test: 3.4482934 i: 2
******************Train: 0.84183854 test: 4.1167474 i: 3
******************Train: 0.71602714 test: 4.629122 i: 4
******************Train: 0.65721637 test: 4.9901714 i: 5
******************Train: 0.62598217 test: 5.2901845 i: 6
******************Train: 0.59936595 test: 5.5108213 i: 7
******************Train: 0.5155404 test: 5.765572 i: 8
******************Train: 0.45746034 test: 5.967716 i: 9


In [56]:
%reload_ext tensorboard

In [57]:
%tensorboard --logdir "out2" --port 9242

Reusing TensorBoard on port 9242 (pid 8716), started 0:00:05 ago. (Use '!kill 8716' to kill it.)